In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import feather
import gc
from datetime import datetime
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from numba import jit

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
%%time
data = pd.read_feather("finalData.ft")

Wall time: 9.51 s


In [3]:
%%time

features = data.columns.tolist()
features.remove('HasDetections')

labels = np.asarray(data['HasDetections'])
data.drop(['HasDetections'],inplace=True,axis=1)


data = np.asarray(data)

Wall time: 13.8 s


In [5]:
%%time
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.metrics import accuracy_score

folds = KFold(n_splits=3, shuffle=True)
scores=[]
confusionMatrices=[]
bestscore=0
for (train_index, test_index) in tqdm(folds.split(data,labels)):
    xtrain , ytrain = np.insert(data[train_index],obj=0,values=1,axis=1) , labels[train_index]
    xtest , ytest = np.insert(data[test_index],obj=0,values=1,axis=1) , labels[test_index]
    
    classifier = LogisticRegression()
    classifier.fit(xtrain , ytrain)
    
    pred = classifier.predict(xtest)
    score = roc_auc_score(ytest, pred)
    scores.append(score)
    confusionMatrices.append(confusion_matrix(ytest,pred))
    if(score>bestscore):
        bestscore=score
        joblib.dump(classifier, 'logRegression.pkl')  
    
print (scores)

0it [00:00, ?it/s]

KeyboardInterrupt: 